In [1]:
import pandas as pd
import numpy as np
import talib

from plotly.subplots import make_subplots

import plotly.graph_objects as go

In [33]:
def assignFactor(filename, training=True):
    df = pd.read_csv(filename)

    df.Date = pd.to_datetime(df.Date)

    df.sort_values(by='Date', inplace=True)
    
    df['sma_15'] = talib.SMA(df['Adj Close'], timeperiod=15)

    df['sma_15_diff'] = df['sma_15'].diff()
    
    df['Trend'] = 'no'
    
    start_ = min(df.index) + 20

    for d, row in df[df.index > start_].iterrows():
        if sum(df[(df.index <= d) & (df.index > d - 5)].sma_15_diff < 0) == 5 and row['sma_15'] > row['Adj Close']:
            df.loc[d, 'Trend'] = 'down'
        elif sum(df[(df.index <= d) & (df.index > d - 5)].sma_15_diff > 0) == 5 and row['sma_15'] < row['Adj Close']:
            df.loc[d, 'Trend'] = 'up'
            
    df['Trading Signal'] = 0
    
    hold = 0
    
    for d, row in df[df.index > start_].iterrows():
        try:
            min_ = min(df[(df.index >= d) & (df.index < d + 3)]['Adj Close'])
            max_ = max(df[(df.index >= d) & (df.index < d + 3)]['Adj Close'])
        except:
            print("Can't lood ahead")
            break
        if max_ != min_:
            if row['Trend'] == 'up':
                hold = 0.5
                df.loc[d, 'Trading Signal'] = (row['Adj Close'] - min_)*0.5/(max_ - min_) + hold

            elif row['Trend'] == 'down':
                hold = 0
                df.loc[d, 'Trading Signal'] = (row['Adj Close'] - min_)*0.5/(max_ - min_) + hold
            else:
                df.loc[d, 'Trading Signal'] = (row['Adj Close'] - min_)*0.5/(max_ - min_) + hold
                
    macd, macdsignal, macdhist = talib.MACD(df['Adj Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['macd'] = macd
    df['macdsignal'] = macdsignal
    
    slowk, slowd = talib.STOCH(df.High.values, df.Low.values, df.Close.values, 5, 3, 0, 3, 0)

    df['slowk'], df['slowd'] = slowk, slowd
    
    df['RSI'] = talib.RSI(df['Adj Close'])
    
    df['WILLR'] = talib.WILLR(df.High.values, df.Low.values, df.Close.values)
    
    df.dropna(inplace=True)
    
#     if training:
#         return df[(df.Date < '2018-01-01') ]
#     else:
#         return df[(df.Date >= '2018-01-01') ]
    return df
    
import numpy as np
import time


def _mean_squared_error(y, pred):
    return 0.5 * np.mean((y - pred) ** 2)


def _mean_abs_error(y, pred):
    return np.mean(np.abs(y, pred))


def _sigmoid(x):
    return 1. / (1. + np.exp(-x))

def _tanh(x):
    return np.tanh(x)

def _fourier(x):
    return np.sin(x)


def _hardlimit(x):
    return (x >= 0).astype(int)


def _identity(x):
    return x


def getActivation(name):
    return {
        'sigmoid': _sigmoid,
        'fourier': _fourier,
        'hardlimit': _hardlimit,
        'tanh': _tanh
    }[name]


def getLoss(name):
    return {
        'mse': _mean_squared_error,
        'mae': _mean_abs_error
    }[name]


class ELM:
    def __init__(self, num_input_nodes, num_hidden_units, num_out_units, activation='sigmoid',
                 loss='mse', beta_init=None, w_init=None, bias_init=None):
        self._num_input_nodes = num_input_nodes
        self._num_hidden_units = num_hidden_units
        self._num_out_units = num_out_units

        self._activation = getActivation(activation)
        self._loss = getLoss(loss)

        if isinstance(beta_init, np.ndarray):
            self._beta = beta_init
        else:
            self._beta = np.random.uniform(-1., 1., size=(self._num_hidden_units, self._num_out_units))

        if isinstance(w_init, np.ndarray):
            self._w = w_init
        else:
            self._w = np.random.uniform(-1, 1, size=(self._num_input_nodes, self._num_hidden_units))

        if isinstance(bias_init, np.ndarray):
            self._bias = bias_init
        else:
            self._bias = np.zeros(shape=(self._num_hidden_units,))

        print('Bias shape:', self._bias.shape)
        print('W shape:', self._w.shape)
        print('Beta shape:', self._beta.shape)

    def fit(self, X, Y, display_time=False):
        H = self._activation(X.dot(self._w) + self._bias)

        # Moore–Penrose pseudo inverse
        if display_time:
            start = time.time()
        H_pinv = np.linalg.pinv(H)
        if display_time:
            stop = time.time()
            print(f'Train time: {stop-start}')

        self._beta = H_pinv.dot(Y)

        # print('Fit Beta shape:', self._beta.shape)

    def __call__(self, X):
        H = self._activation(X.dot(self._w) + self._bias)
        return H.dot(self._beta)

    def evaluate(self, X, Y):
        pred = self(X)

        # Loss (base on model setting)
        loss = self._loss(Y, pred)

        # Accuracy
        acc = np.sum(np.argmax(pred, axis=-1) == np.argmax(Y, axis=-1)) / len(Y)

        # Unweighted Average Recall
        # TODO

        return loss, acc

In [34]:
from sklearn.preprocessing import MinMaxScaler

In [35]:
import pickle

In [36]:
model = pickle.load(open("ELM.pickle", "rb", -1))

scaler = pickle.load(open('sclar.pickle', 'rb', -1))

In [37]:
df_test = assignFactor('/home/usman/Downloads/ATVI.csv', training=False)

In [38]:
test = df_test[['sma_15', 'macd', 'macdsignal', 'slowk', 'slowd', 'RSI', 'WILLR']]

X_test = scaler.transform(test)

y_test = df_test['Trading Signal'].values

In [39]:
pred = model.__call__(X_test)

0.5 * np.mean((y_test - pred) ** 2)

0.04429783902243624

In [56]:
df_test['pred'] = pred

def buysellSignal(x):
    if x > 0.8:
        return 'SELL'
    if x < 0.2:
        return 'BUY'
    
df_test['action'] = df_test['pred'].apply(lambda x: buysellSignal(x))

In [57]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_test['Date'], y=df_test['Adj Close'],
                    mode='lines',
                    name='markers'))

fig.add_trace(go.Scatter(x=df_test[df_test['action'] == 'SELL'].Date, y=df_test[df_test['action'] == 'SELL']['Adj Close'],
                    mode='markers',
                    name='SELL'))

fig.add_trace(go.Scatter(x=df_test[df_test['action'] == 'BUY'].Date, y=df_test[df_test['action'] == 'BUY']['Adj Close'],
                    mode='markers',
                    name='BUY'))


fig.show()